## Text preparation 

build a vocabulary.

In [ ]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [ ]:
# Mounted Drive if using Colab; otherwise, your local path
root = "/drive/My Drive/Colab Notebooks/Flickr8k/" # <--- replace this with your root data directory
#/drive/My Drive/Colab Notebooks/Flickr8k/Flickr8k_Dataset
caption_dir = root + "Flickr8k_text/"                      # <--- replace these too
image_dir = root + "Flickr8k_Dataset/"                            # <---
#/drive/My Drive/Colab Notebooks/Flickr8k/Flickr8k_text/Flickr8k.token.txt

token_file = "Flickr8k.token.txt"

A helper function to read in our ground truth text file.

In [ ]:
def read_lines(filepath):
    """ Open the ground truth captions into memory, line by line. """
    file = open(filepath, 'r')
    lines = []

    while True: 
        # Get next line from file until there's no more
        line = file.readline() 
        if not line: 
            break
        lines.append(line.strip())
    file.close() 
    return lines

read all the ground truth captions (5 per image), into memory as follows:

In [ ]:
lines = read_lines(caption_dir + token_file)

In [ ]:
len(lines)

40455

In [ ]:
file = open(caption_dir + token_file, 'r') #remove 
doc = file.read()

In [ ]:
def load_captions(doc):
    mapping = dict()
# process lines
    for line in doc.split('\n'):
# split line by white space
        tokens = line.split()
        if len(line) < 2:
            continue
# take the first token as the image id, the rest as the description
        image_id, image_desc = tokens[0], tokens[1:]
# extract filename from image id
        image_id = image_id.split('.')[0]
# convert description tokens back to string
        image_desc = ' '.join(image_desc)
# create the list if needed
        if image_id not in mapping:
           mapping[image_id] = list()
# store description
           mapping[image_id].append(image_desc)
    return mapping

# parse descriptions
captions_dict = load_captions(doc)
#print('Loaded: %d ' % len(descriptions))

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Lemmatization?
Lemmatization is not necessary. We want to compare performance on different decoder normal RNN and LSTM.  It is probably better to control other factors that may affect performance of these two models if we want a better comparison. The main reason of using lemmatization is increasing accuracy of generating caption, so if we want to develop a better model to obtain more accurate caption, we should consider using lemmatized token.


In [ ]:
import nltk
import string
from collections import Counter
counter=Counter()
tokens=[]
filtered_captions_dict = {}
#ids=captions_dict.keys
for image, captions in captions_dict.items():
    filtered_captions_dict[image] = []
    for caption in captions:
        caption = caption.translate(str.maketrans('', '', string.punctuation))
        caption=str(caption)
        caption=caption.lower()
        tokens.extend(nltk.tokenize.word_tokenize(caption))
        filtered_captions_dict[image].append(caption.split())
  
    counter.update(tokens)


# If the word frequency is less than 'threshold', then the word is discarded.
#words= [word for word in words if word.isalnum()]
words = [word for word, cnt in counter.items() if cnt > 3]

In [ ]:
unique_filtered_captions_dict= {}
cleaned_captions = []
image_ids = [] 
for key, values in filtered_captions_dict.items():
    for caption in values:
        cleaned_caption_joined = None
        image_ids.append(key)
    for word in caption:
        if word in words:
        cleaned_caption_joined = cleaned_caption_joined + " " + word if cleaned_caption_joined else word
    cleaned_captions.append(cleaned_caption_joined)

captions_dict.values()

In [ ]:
class Vocabulary(object):
    """Simple vocabulary wrapper which maps every unique word to an integer ID. """
    def __init__(self):
        # Intially, set both the IDs and words to empty dictionaries.
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        # If the word does not already exist in the dictionary, add it
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            # Increment the ID for the next word
            self.idx += 1

    def __call__(self, word):
        # If we try to access a word in the dictionary which does not exist, return the <unk> id
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

Extract all the words from ```lines```, and create a list of them in a variable ```words```, for example:

```words = ["a", "an", "the", "cat"... ]```

No need to worry about duplicates.


Build the vocabulary.

In [ ]:
# Create a vocab instance
vocab = Vocabulary()

# Add the token words first
vocab.add_word('<pad>')
vocab.add_word('<start>')
vocab.add_word('<end>')
vocab.add_word('<unk>')

for i, word in enumerate(words):
    vocab.add_word(word)

Add the rest of the words from the parsed captions:

``` vocab.add_word('new_word')```

Don't add words that appear three times or less.

In [ ]:
import pickle
vocab_path=F'/drive/My Drive/Colab Notebooks/Flickr8k/vocab.pkl'
vocab_path = vocab_path
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab, f)
print("Total vocabulary size: {}".format(len(vocab)))

Total vocabulary size: 3440
